In [1]:
from google.cloud import bigquery
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
import numpy as np
import requests
from datetime import datetime as dt
from datetime import date

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\creds\opendatadbt-f2e674527eb6.json'
client = bigquery.Client()

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'.\creds\opendatadbt-f2e674527eb6.json') # Your json file here


#Initial Load boston_table_id = "opendatadbt.311.boston311"

sf311 = requests.get("https://data.sfgov.org/resource/vw6y-z8j6.json?$limit=50000&$where=requested_datetime>'2021-03-24'" ) 
sf311json = sf311.json()
cols = ['service_request_id','requested_datetime','closed_date','agency_responsible','service_name','service_details','address','neighborhoods_sffind_boundaries','status_description','lat','long']
sf_df = pd.DataFrame(sf311json, columns=cols)
sf_df

In [3]:
#Incremental load
sf_table_id = "opendatadbt.311.sf311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
max(requested_datetime), max(closed_date)
FROM `opendatadbt.311.sf311`
limit 10
"""

sfmaxopendatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
sfmaxopendatedate = str(sfmaxopendatedataframe['f0_'][0])

sfmaxclosedatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
sfmaxclosedate = str(sfmaxclosedatedataframe['f1_'][0])


In [4]:

sf311 = requests.get("https://data.sfgov.org/resource/vw6y-z8j6.json?$limit=50000&$where=requested_datetime>'"+sfmaxopendatedate+"' OR closed_date between '" + sfmaxclosedate+ "' AND '" + current_datetime + "'") 
sf311json = sf311.json()
cols = ['service_request_id','requested_datetime','closed_date','agency_responsible','service_name','service_details','address','neighborhoods_sffind_boundaries','status_description','lat','long']
sf_df = pd.DataFrame(sf311json, columns=cols)
sf_df

,service_request_id,requested_datetime,closed_date,agency_responsible,service_name,service_details,address,neighborhoods_sffind_boundaries,status_description,lat,long
0,11647451,2019-11-05T08:35:00.000,2021-03-26T12:00:34.000,City Attorney - G,General Request - CITY ATTORNEY,city_attorney - customer_callback,Not associated with a specific address,NaN,Closed,0E-12,0E-12
1,12840324,2020-08-31T05:48:00.000,2021-03-26T12:00:35.000,City Attorney - G,General Request - CITY ATTORNEY,city_attorney - customer_callback,Not associated with a specific address,NaN,Closed,0E-12,0E-12
2,13591136,2021-03-13T12:53:00.000,2021-03-26T22:07:58.000,Muni Feedback Received Queue,Muni Employee Feedback,Waiting_At_Stop,Intersection of SANSOME ST and SUTTER ST,Financial District,Closed,37.790247387678,-122.400555212485
3,13594720,2021-03-15T06:34:02.000,2021-03-26T21:30:36.000,Clear Channel - Transit Queue,Graffiti,Transit_Shelter_Platform - Not_Offensive,"599 OCEAN AVE, SAN FRANCISCO, CA, 94112",Cayuga,Closed,37.72270471,-122.44714949
4,13594884,2021-03-15T07:57:00.000,2021-03-26T11:07:00.000,SFMTA - Parking Enforcement - G,Parking Enforcement,White - Isuzu box truck - 5w61331,"1439 SAN BRUNO AVE, SAN FRANCISCO, CA, 94110",Potrero Hill,Closed,37.75053138,-122.40395721
...,...,...,...,...,...,...,...,...,...,...,...
2413,13620910,2021-03-22T13:12:00.000,2021-03-26T07:49:00.000,SFMTA - Parking Enforcement - G,Parking Enforcement,All - All - All,"19 CLEMENTINA ST, SAN FRANCISCO, CA, 94105",South of Market,Closed,37.78727881,-122.39557765
2414,13620911,2021-03-22T13:12:26.000,2021-03-26T12:19:00.000,DPW BSSR Queue,Street Defects,Pavement_Defect,"1160 FULTON ST, SAN FRANCISCO, CA, 94117",Alamo Square,Closed,37.777150961004,-122.435862180301
2415,13617400,2021-03-21T13:06:00.000,2021-03-26T15:49:18.000,Muni Feedback Received Queue,Muni Employee Feedback,Discourteous_Rude,Not associated with a specific address,NaN,Closed,0E-12,0E-12
2416,13622126,2021-03-22T16:51:59.000,2021-03-26T07:23:00.000,SFMTA - Parking Enforcement - G,General Request - MTA,parking_enforcement - request_for_service,Not associated with a specific address,NaN,Closed,0E-12,0E-12


In [5]:
sf_table_id = "opendatadbt.311.sf311"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('sf311')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client.load_table_from_dataframe(
    sf_df, table_ref, job_config=job_config
).result()  # Make an API request.
